# Stability calculations

## -- LUVOIR --

In notebook 10, we calculated the PASTIS modes for all three currently designed apodizers for the LUVOIR A pupil. In this notebook, we will load the modes and calculate the maximum mode contributions $\sigma$ and strability requirements $\Delta \sigma$.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
from luvoir_imaging import LuvoirAPLC

## Instantiate a simple SM for mode plotting

I need to simplify this... I can make a child class like I did for the full LUVOIR simulator.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(poslist)

In [ ]:
# Instantiate SM
sm = SegmentedMirror(aper_ind, seg_pos)

## Mode and eigenvalue units

Since the matrices got generated normalized to an aberration of 1 nm, everything stemming from the matrix will be in units of **nanometers**. --> This is the desired behaviour of course and is yet to be confirmed, which is one of the goals of this notebook.

In [ ]:
eunit = 1e-9

## Also instantiate a LUVOR telescope

For cumulative contrast calculation.

> **ATTENTION:** the variable `apodizer_design` has to be correct for the data we are reading with the variable `savedpath` further below!!!

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'small'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf/norm, norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf/norm, norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf/norm * dh_mask
baseline_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', baseline_contrast)

## Read eigenmodes and eigenvalues

In [ ]:
# Which directory are we working in?
savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-5-31_002_1nm'

# Load eigenvalues - lowest first, highest at the end
evals = np.loadtxt(os.path.join(savedpath, 'results', 'eigenvalues.txt'))
print('evals.shape: {}'.format(evals.shape))

# Load eigenmodes - piston value per segment per mode
emodes = np.loadtxt(os.path.join(savedpath, 'results', 'eigenvectors.txt'))
print('emodes.shape: {}'.format(emodes.shape))

# Load eigenmodes (just for plotting) - mode images
epics = hc.read_fits(os.path.join(savedpath, 'results', 'modes', 'fits', 'cube_modes.fits'))
print('emodes.shape: {}'.format(epics.shape))

Check that eigenmodes are in correct order and how to address them in the data cube.

In [ ]:
mode = -1   # We start numbering at 0 here, 0-35 (Python nunmbering!)

sm.flatten()
for seg, val in enumerate(emodes[:, mode]):
    #print(val)
    sm.set_segment(seg+1, eunit*val/2, 0, 0)

# Propagate WF and display SM phase
wf_sm = sm(wf_aper)

hc.imshow_field(wf_sm.phase, cmap='RdBu')
plt.colorbar()

So we have `emodes[segments, modes]`.

## Stability requirements

### Static contrast and static contribution

According to Lucie's paper, we can get the maximum aberration $\sigma_p$ we can allow per mode $p$ if we want to obtain a contrast $C$ directly from the according eigenvalue $\lambda_p$.

$$\sigma_p = \sqrt{\frac{C_p}{\lambda_p}}$$

Where $C_p$ is the contrast contribution from mode $p$ only. For simplicity, we assume that all modes have the same contribution to the total contrast $C$, meaning

$$C = C_1 + C_2 + ... + C_N$$

with $N$ being the total number of modes. This also means

$$C_p = \frac{C}{N}$$

Since we have one mode that has a really low eigenvalue and essentially on contribution, we can take that mode out and assume that the other $N-1$ modes bear all the contrast contributions, changing it to

$$C_p = C_1 + C_2 + ... + C_{N-1}$$

and

$$C_p = \frac{C}{N-1}$$

So we can calculate the maximum ocntribution of a mode $\sigma_p$ with

$$\sigma_p = \sqrt{\frac{C}{(N-1)\lambda_p}}$$

We have 120 segments, so $N=120$ and we will aim to achieve a static contrast of $C = 10^{-10}$.

Note how we start numbering at 1 here, to stay consistent with our segment numbering, but when doing these things in Python we need to start numbering at 0.

Skype with Lucie, have to include baselin contrast $C_0$:

$$\sigma_p = \sqrt{\frac{C-C_0}{(N-1)\lambda_p}}$$

In [ ]:
c_stat = 1e-10
print('Static contrast: {}'.format(c_stat))
print('N = {}'.format(nseg))

In [ ]:
# Calculate single sigma - remember that we start numbering at 0 because of python
def get_sigma(cstat, nseg, eigenval, c_zero):
    sigma = np.sqrt((cstat - c_zero) / ((nseg-1)*eigenval))
    return sigma

In [ ]:
p = 1
sigma_p = get_sigma(c_stat, nseg, evals[p], baseline_contrast)
print(sigma_p)

In [ ]:
# Do them all at once
sigmas = get_sigma(c_stat, nseg, evals, baseline_contrast)
print(sigmas)

In [ ]:
plt.plot(sigmas)
plt.title('Constraints per mode')
plt.xlabel('Mode')
plt.ylabel('Max mode contribution $\sigma_p$ (nm)')

### Dynamic contast and dynamic contribution

We want $\Delta C = 10^{-11}$. This "dynamic contast" is the error on the contrast $C$ which we want to limit, so we write it as

$$\Delta C = \sqrt{\Delta C_1^2 + \Delta C_2^2 + ... \Delta C_N^2} = \sqrt{N \Delta C_p^2} = \sqrt{N} \Delta C_p$$

which also means

$$\Delta C_p = \frac{C}{\sqrt{N}}$$

and then again because we discard that non-contributing mode, we actually have

$$\Delta C_p = \frac{C}{\sqrt{N-1}}$$

For the $\Delta \sigma_p$ we can use the same equation like for the $\sigma_p$, but we plug in $\Delta C$ instead of $C$.

$$\Delta \sigma_p = \sqrt{\frac{\Delta C}{\lambda_p}} = \sqrt{\frac{\Delta C}{\sqrt{(N-1)}\ \lambda_p}}$$

In [ ]:
c_dyn = 1e-11

# Calculate the Delta Cs
def get_delta_sigma(cdyn, nseg, eigenval):
    del_sigma = np.sqrt(cdyn / (np.sqrt(nseg-1)*eigenval))
    return del_sigma

In [ ]:
del_sigmas = get_delta_sigma(c_dyn, nseg, evals)
print(del_sigmas)

In [ ]:
plt.plot(del_sigmas)
plt.title('Stability per mode')
plt.xlabel('Mode')
plt.ylabel('Max mode contribution $\Delta \sigma_p$ (nm)')

These are only slightly larger than theh $\sigma_p$.

### Cumulative contrast plot

I guess the only way I can think of right now to verify at least the $\sigma_p$ is to make the same cumulative contrast like Fig. 11b in Lucie's paper. Let's do that.

Since I never get rid of the first mode, global piston with a ridiculously low eigenvalue, I will have a `Nan` in the sigma array, so when I sum up the OPD, I have to use `numpy.nansum()` instead of `numpy.nan()`.

In [ ]:
cont_cuml = []
for maxmode in range(len(evals)):

    opd = np.nansum(emodes[:, :maxmode+1]*sigmas[:maxmode+1], axis=1)

    luvoir.flatten()
    for seg, val in enumerate(opd):
        luvoir.set_segment(seg+1, eunit*val/2, 0, 0)
        
    # Get PSF from putting this OPD on the SM
    psf = luvoir.calc_psf()

    # Calculate the contrast from that PSF
    dh_intensity = psf/norm * dh_mask
    contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
    cont_cuml.append(contrast)

In [ ]:
# Just having a look at the last PSF from that loop above
hc.imshow_field(np.log10(psf))

In [ ]:
# Plot the cumulative contrast
plt.plot(cont_cuml)
plt.title('Cumulative contrast')

Sweeeeeet!!! :D I realized today that I have been using the medium apodizer design all along, but then remembered that form the three LUVOIR APLC designs, only the small one had a good accordance of its contrast calculation with the matrix compared with the E2E simulation - the medium and large designs still have a constant offset that I haven't figured out. So when I reverted back to using the small design, this essentially fell out of the sky haha.

## Mode rms

Let's check whether the  modes are normalized to the correct overall rms and if not, what their units are.

In [ ]:
rms_modes = []
for mod in range(len(evals)):
    rms = np.std(emodes[:, mod])
    rms_modes.append(rms)

In [ ]:
rms_modes

They're off by a factor of 10. Or do I have to multipy them by their sigmas?

In [ ]:
# Plot the RMS per mode, excluding first, global piston mode
plt.plot(rms_modes[1:])

In [ ]:
rms_sigma = []
for mod in range(len(evals)):
    rms = np.std(emodes[:, mod] * sigmas[mod])
    rms_sigma.append(rms)

In [ ]:
rms_sigma

Nope, definitely not.